In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import calendar
import datetime
import json 
import os  
import pickle
from configparser import ConfigParser
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

### General helper functions

In [2]:
# format release date
def format_release_date(data):
    arr = list()
    for i,l in enumerate(data):
        if ord(l) == 45:
            arr.append(i)
    date = int(data[0:arr[0]])
    month_name = data[arr[0]+1:arr[1]]
    datetime_object = datetime.datetime.strptime(month_name, "%b")
    month = datetime_object.month
    year = int(data[arr[1]+1:arr[1]+5])
    date = datetime.datetime(year, month, date)
    utc_time = calendar.timegm(date.utctimetuple())
    return utc_time 

# format timestamp
def date_only_from_datetime(data):
    temp = datetime.datetime.fromtimestamp(data)
    date = datetime.datetime(temp.year, temp.month, temp.day) # temp.hour
    return date

# get mean rating for the user x
def get_mean_rating(x, mean_rating):
    return float(mean_rating[mean_rating['user_id'] == x]['mean_rating'].to_numpy()[0])

In [3]:
#Read config.ini file
config = ConfigParser()
config.read("config.ini")
fileinfo = config["DATASETS"]

path = fileinfo['ori_path']
fm_export_path = fileinfo['fm_path']
rf_export_path = fileinfo['rf_path']
train_set_filename = fileinfo['train_filename']
test_set_filename = fileinfo['test_filename']
description_filename = fileinfo['description_filename']

# import user, data, item and genre data
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(path + 'ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1', parse_dates=True, header=None) 
d_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
data = pd.read_csv(path + 'ml-100k/u.data', sep='\t', names=d_cols, encoding='latin-1', header=None)
genre_cols = pd.read_csv(path + 'ml-100k/u.genre', sep='|', header=None)[0].to_numpy().tolist()
m_cols = ['movie_id', 'movie_title', 'release_date', 'video_release_date','imdb_url']
m_cols.extend(genre_cols)
movies = pd.read_csv(path + 'ml-100k/u.item', sep='|', names=m_cols, usecols=range(24), encoding='latin-1', header=None)

# merge user, data and item
movie_ratings = pd.merge(movies, data)
df = pd.merge(movie_ratings, users)

### Before split - general

- These are columns that need to be preprocessed for both FM and RF, but are don't need to be split beforehand
- Columns: 
  - release_date
  - sex
  - zip_code

In [4]:
# dropping some rows that has no release date 
df.drop(index = df[df['release_date'].isnull() == True].index, inplace = True)
df.reset_index(drop=True, inplace=True)

# format release_date to unix timestamp
df['release_date'] = df['release_date'].apply(lambda x: format_release_date(x))

# convert to binary
df['sex'] = (df['sex'] == 'M').astype(int)

# take first character only
df['zip_code'] = df['zip_code'].str[0]

### Before split - columns for FM

- These are columns that need preprocessing specific for FM, but don't have to be split beforehand
- Columns:
  - age
  - timestamp
  - release_date
- In addition, the one-hot-encoders are fitted here, before splitting

In [5]:
# format age
# age_labels = []
# age_bins = np.arange(0,90,10)
# for i in range(len(age_bins)-1):
#     age_labels.append(str(age_bins[i]) + '-' + str(age_bins[i+1]))
# # convert ages to group of age ranges
# df['fm_age'] = pd.cut(df['age'], bins = age_bins, labels=age_labels)

In [6]:
# preprocess timestamp and release date
# only use year, month and day from the datetime
le = preprocessing.LabelEncoder()
df['timestamp'] = df['timestamp'].apply(lambda x : date_only_from_datetime(x))
df['timestamp'] = le.fit_transform(df['timestamp'])
df['release_date'] = le.fit_transform(df['release_date'])
# df['release_date'] = df['release_date']
# df['fm_release_date'] = df['release_date'].apply(lambda x : date_only_from_datetime(x))

In [7]:
# fit one-hot-encoders before train/test split
ohe_columns = ["movie_id", "user_id", "zip_code", "occupation"]
ohe_encoders = []
for col in ohe_columns:
    encoder = OneHotEncoder(dtype=int, sparse=False)
    encoder.fit(df[[col]])
    ohe_encoders.append(encoder)

### Before split - columns for RF

- These are columns that need preprocessing specific for RF, but don't need to be split beforehand
- Columns: none at the moment.

### Train/Test split

In [8]:
# Splitting the data into test and train
X_train, X_test, y_train, y_test = train_test_split(df.drop(['rating'], axis = 1), df['rating'], random_state=42)

# training and test data is combined to a single dataframe
df_train = pd.concat([X_train,pd.DataFrame(y_train)], axis = 1)
df_test = pd.concat([X_test,pd.DataFrame(y_test)], axis = 1)

del(df)

### After split - general

- These are columns that have to be preprocessed after the split.
- Columns:
  - mean_rate
  - cum_mean_rate
  - last_watched

In [9]:
# get last watched movie
def get_last_watched(df):
    df.sort_values(['user_id', 'timestamp'], inplace=True)
    # shift the values in movie_id col down by one
    df['last_watched'] = df['movie_id'].shift(1)
    # then set the first value (first movie watched by the user) to 0
    for index, user in users.iterrows():
        idx = df[df['user_id'] == user['user_id']].index[0]
        df.at[idx, 'last_watched'] = df.at[idx, 'movie_id']

In [10]:
# for df in [df_train, df_test]:
#     get_last_watched(df)

In [11]:
# Get mean and cumulative mean
# it is only done for training data, since we should not take statistics about rating from test data
data_train = df_train[['user_id','movie_id','rating','timestamp']]

# calculating cumulative mean rating based on timestamp sorted in ascending order
data_train = data_train.sort_values(['user_id', 'timestamp']).reset_index(drop=True)
data_train['one'] = 1
data_train['cumsum'] = data_train.groupby('user_id')['one'].cumsum()
data_train['cum_mean_rate'] = data_train.groupby('user_id')['rating'].cumsum() / data_train['cumsum']

# calculating mean rating for each user
mean_rating = data_train.groupby('user_id')['rating'].sum() / data_train.groupby('user_id')['one'].sum()
mean_rating = mean_rating.reset_index()
mean_rating.rename(columns = {0:'mean_rating'}, inplace = True)

# getting mean rate for the corresponding user
data_train['mean_rate'] = data_train['user_id']
data_train["mean_rate"] = data_train['mean_rate'].apply(lambda x: get_mean_rating(x, mean_rating))

# merging to the original dataframe
df_train = df_train.merge(data_train, how='left', left_on=['timestamp', 'user_id', 'movie_id', 'rating'], right_on=['timestamp', 'user_id', 'movie_id', 'rating'])

# during testing we won't have the rating value, so we should use the running mean form training data.
df_test['mean_rate'] = df_test['user_id'].apply(lambda x: get_mean_rating(x, mean_rating))
df_test['cum_mean_rate'] = df_test['mean_rate']

del(data_train)

In [12]:
# RF forest data needs to be preprocessed differently. 
# Therefore need to save copy of dfs first
rf_df_train = df_train.copy(deep=True)
rf_df_test = df_test.copy(deep=True)

### After split - FM

- These are steps that need to be performed after the split, and specifically for FM
- Steps:
  - OHE and other encoding
  - Getting the feature column lengths (col_len)
  - Saving the data
- columns used in FM:
  - movie_id, user_id, timestamp, release_date, zip_code, age, occupation, genre, sex, rating
  - TODO: binary encoding last_watched, maybe?

In [13]:
# ohe_columns[-1] = "last_watched"
# ohe_encoders[-1].feature_names_in_ = [ohe_columns[-1]]

In [14]:
# add mean_rate and cum_mean_rate to encoder
# for col in ['mean_rate', 'cum_mean_rate']:
#     df_train[col] = df_train[col].apply(lambda rate: int(round(rate, 1)*10))
#     df_test[col] = df_test[col].apply(lambda rate: int(round(rate, 1)*10))
#     ohe_columns.append(col)
#     encoder = OneHotEncoder(categories=[[i for i in range(10,51,1)]], dtype=int, sparse=False)
#     encoder.fit(df_train[[col]])
#     ohe_encoders.append(encoder)

In [15]:
# create the final df and set col_len at the same time
def encode_df(df, col_len, ohe_cols, ohe_encoders):
    # create empty df with correct shape
    df_final = pd.DataFrame(columns = [], index = range(len(df)))
    
    # columns that need ohe
    for index, col in enumerate(ohe_cols):
        transform = pd.DataFrame(ohe_encoders[index].transform(df[[col]]), columns=ohe_encoders[index].get_feature_names_out())
        df_final = pd.concat([df_final, transform], axis = 1)
        col_len[col] = len(transform.columns)

    # special case: genre (already ohe in original dataset)
    col_len["genre"] = len(genre_cols)
    df_final = pd.concat([df_final, df[genre_cols]], axis = 1)

    # columns that don't need ohe
    non_ohe_columns = ["sex", "age", "timestamp", "release_date", "mean_rate", "cum_mean_rate"]
    for col in non_ohe_columns:
        col_len[col] = 1
    df_final = pd.concat([df_final, df[non_ohe_columns]], axis = 1)

    # rating column (no need to add to col_len)
    df_final = pd.concat([df_final, df[['rating']]], axis = 1)

    return df_final

In [16]:
col_len = {}

#need to reset otherwise pd.concat will not work properly
df_test.reset_index(inplace=True)
df_train.reset_index(inplace=True)

df_test = encode_df(df_test, col_len, ohe_columns, ohe_encoders)
df_train = encode_df(df_train, col_len, ohe_columns, ohe_encoders)

In [25]:
df_train

,movie_id,user_id,timestamp,release_date,zip_code,age,occupation,sex,rating,mean_rate,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,132,795,96,11,0,30,14,1,3,3.108527,...,0,0,0,1,0,0,0,0,0,0
1,429,499,124,23,7,42,14,1,4,3.855422,...,0,0,0,0,0,0,1,0,0,0
2,106,839,12,81,9,38,5,0,2,3.078947,...,0,0,0,0,0,0,0,1,0,0
3,341,7,202,171,9,57,0,1,3,3.970684,...,0,0,0,0,0,0,0,0,0,0
4,145,773,158,72,5,20,18,1,3,3.336066,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74988,666,279,180,49,8,33,14,1,2,3.291536,...,0,0,1,0,0,0,0,0,0,0
74989,282,294,36,123,9,34,19,1,3,3.403670,...,0,0,0,0,0,0,0,0,0,0
74990,708,524,112,69,0,56,3,1,4,3.465021,...,0,0,0,0,0,0,0,0,0,0
74991,167,5,10,57,1,33,13,0,2,2.831933,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Saving the FM data
# create parent folder if doesn't exist
# os.makedirs(fm_export_path, exist_ok=True)
# with open(fm_export_path + "fm_train.csv", "w") as f:
#     print(f)
#     df_train.to_csv(f)

# with open(fm_export_path + "fm_test.csv", "w") as g:
#     print(g)
#     df_test.to_csv(g)



# export csv file
pickle.dump(df_train, open(fm_export_path + train_set_filename, 'wb'))
pickle.dump(df_test, open(fm_export_path + test_set_filename, 'wb'))

# saving info about data
with open(fm_export_path + description_filename, "w") as outfile:
    json.dump(col_len, outfile)


### After split - RF

- These are steps that need to be performed after the split, and specifically for RF
- Steps:
  - label encoding
  - Saving the data
- Columns used in RF:
  - movie_id, user_id, timestamp, release_date, zip_code, age, 
  occupation, sex, rating, last_watched, mean_rate, cum_mean_rate

In [21]:
final_columns = ['movie_id', 'user_id', "timestamp", "release_date", "zip_code", "age", 
"occupation", "sex", "rating", "mean_rate", "cum_mean_rate"]
final_columns.extend(genre_cols)
# droping all unnecessary columns
df_train = rf_df_train[final_columns]
df_test = rf_df_test[final_columns]

del(rf_df_train)
del(rf_df_test)

In [22]:
# converting objects to integers, since scikit learn supports only numerical values
le = preprocessing.LabelEncoder()

label_encoding_columns = ['occupation','zip_code','sex']
for col in label_encoding_columns:
    df_train[col] = le.fit_transform(df_train[col])
    df_test[col] = le.fit_transform(df_test[col])    

<ipython-input-22-08813cd8c4b1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[col] = le.fit_transform(df_train[col])


In [24]:
df_train.columns

Index(['movie_id', 'user_id', 'timestamp', 'release_date', 'zip_code', 'age',
       'occupation', 'sex', 'rating', 'mean_rate', 'cum_mean_rate', 'unknown',
       'Action', 'Adventure', 'Animation', 'Children's', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'],
      dtype='object')

In [ ]:
# create parent folder if doesn't exist
os.makedirs(rf_export_path, exist_ok=True)

# Saving the RF data
pickle.dump(df_train, open(rf_export_path + train_set_filename, 'wb'))
pickle.dump(df_test, open(rf_export_path + test_set_filename, 'wb'))
